In [1]:
import csv
import pandas as pd
from os import listdir

from files_def_multiple_runs import *

In [2]:
dir_run = '../results'

In [3]:
def generate_all_results_best_results_and_best_configuration(speeds, execution_folder, solver_name, Npop, Mit):
    
    for s in speeds:
        speed = s.replace('_', '.')
        cmaes_dir = dir_run  + '/' + execution_folder + '/' + s
        
        best_result = {
            'D':0,
            'AEdAO':0,
            'PdD':0,
            'Z':0,
            'P_B':0,
            'fitness':-999,
            'seed':0,
            'iteration':0
        }
        
        for seed in range(0,10):
            data = []
            
            seed_dir = cmaes_dir + "/" + str(seed)
            for Z in range(2,8):
                
                Z_file = seed_dir + "/" + str(Z) + ".csv"
                
                with open(Z_file, 'r', newline='') as csv_file:
                    reader = csv.reader(csv_file)
                    counter = 1
                    iteration = 0
                    for row in reader:
                        
                        if (counter == 1):
                            counter += 1
                            continue
                        
                        if (row[0] == 'fitness at iteration'):
                            counter += 1
                            iteration += 1
                            continue
                        
                        if (row[0] == 'Best Solution'):
                            break
                        
                        row.append(seed)
                        row.append(iteration)
                        
                        data.append(row)
                        
                        counter += 1
            
            columns = ['D', 'AEdAO', 'PdD', 'Z', 'P_B', 'n', 'fitness', 't075dD', 'tmin075dD', 'tal07R', 'cavLim', 'Vtip', 'Vtipmax', 'penalty', 'valid', 'seed', 'iteration']
                    
            df = pd.DataFrame(data, columns=columns)
            
            df['fitness'] = df['fitness'].astype(float)
            
            df.to_csv(seed_dir + '/' + 'all_results.csv', index=False)
            
            
            # Get valid designs
            valid_df = df.loc[df['valid'] == 'True']
            
            df_sorted = valid_df.sort_values(by='fitness', ascending=False).reset_index(drop=True)
            
            max_fitness_row = df_sorted.iloc[0]
            
            filename = create_file(seed_dir, 'best_results_' + str(seed) +'_'+ solver_name)
            append_to_file_order(filename, D=max_fitness_row['D'], AEdAO=max_fitness_row['AEdAO'], PdD=max_fitness_row['PdD'], Z=max_fitness_row['Z'], P_B=max_fitness_row['P_B'], n=max_fitness_row['n'], fitness=max_fitness_row['fitness'])
            # history
            append_to_file(filename, ['history'])
            append_to_file(filename, "")
            # config
            append_to_file(filename, ['solver Name', 'V_S', 'NPOPULATION', 'MAX_ITERATION', 'SEED'])
            append_to_file(filename, [ solver_name,   speed,   Npop,          Mit,             seed])
            
            if (max_fitness_row['fitness'] > best_result['fitness']):
                best_result['D'] = max_fitness_row['D']
                best_result['AEdAO'] = max_fitness_row['AEdAO']
                best_result['PdD'] = max_fitness_row['PdD']
                best_result['Z'] = max_fitness_row['Z']
                best_result['P_B'] = max_fitness_row['P_B']
                best_result['fitness'] = max_fitness_row['fitness']
                best_result['seed'] = max_fitness_row['seed']
                best_result['iteration'] = max_fitness_row['iteration']
            
        # Save best over all seeds
        with open(cmaes_dir + '/best_configuration.csv', 'w') as csv_file:  
            writer = csv.writer(csv_file)
            for key, value in best_result.items():
                writer.writerow([key, value])
        
        

## Generate files for CMA run

In [4]:
speeds = ["7_0", "7_5", "8_0", "8_5"]
execution_folder = 'cmaes_2024_04_22_16h15'
solver_name = "cmaes"
npopulation = 5
max_iterations = 30

generate_all_results_best_results_and_best_configuration(speeds, execution_folder, solver_name, npopulation, max_iterations)

## Generate files for OpenAI run

In [5]:
speeds = ["7_0", "7_5", "8_0", "8_5"]
execution_folder = 'openaies_2024_04_23_18h02'
solver_name = "openaies"
npopulation = 5
max_iterations = 30

generate_all_results_best_results_and_best_configuration(speeds, execution_folder, solver_name, npopulation, max_iterations)